In [12]:
import requests
import pandas as pd

url = 'http://apis.data.go.kr/6450000/TouristAccoService/getTouristAcco'
params = {'ServiceKey': '',
          'area' : '',
          'category' : '',
        }

mykey = "TEo5fURVg6O3ChvfXOmzkr0IXbTl0d4VkfIj3JVTz0ctJ+NS0IjPHxLXlijxDlubeXvzd3ZlGksTn/HhACp8gA=="
params['ServiceKey']=mykey
# 한 페이지 결과 수
params['area']=input('지역코드 : ')
# 페이지 번호
params['category']=input('숙박종류 : ')


# 오류가 날 때가 있어서(SERVICE KEY ERROR)..그럴 땐 2~3번 실행 하면 됩니다!
response = requests.get(url, params=params)
print(response.content)

b'<response>\n<header>\n<resultCode>200</resultCode>\n<resultMsg>OK</resultMsg>\n</header>\n<body>\n<totalCount>5</totalCount>\n<items>\n<item>\n<IMG>http://www.gojb.net/gwmsData/tourData/LODG_M/p1329724834083-1.jpg</IMG>\n<AREA>12</AREA>\n<SNO>1402</SNO>\n<CATEGORY>01</CATEGORY>\n<OVERVIEW> \xeb\xa5\xb4\xec\x8b\x9c\xec\x97\x98 \xed\x98\xb8\xed\x85\x94\xec\x9d\x80 \xea\xb3\xa0\xea\xb0\x9d \xec\x97\xac\xeb\x9f\xac\xeb\xb6\x84\xeb\x93\xa4\xec\x97\x90\xea\xb2\x8c \xec\x8b\xa0\xeb\xa2\xb0\xea\xb0\x90\xec\x9d\x84 \xec\xa3\xbc\xea\xb8\xb0 \xec\x9c\x84\xed\x95\x98\xec\x97\xac \xec\xa0\x84\xeb\x9d\xbc\xeb\xb6\x81\xeb\x8f\x84\xec\x99\x80 \xec\xa0\x84\xec\xa3\xbc\xec\x8b\x9c\xec\x9d\x98 \xec\xa7\x80\xec\xa0\x95(\xec\xa0\x84\xeb\x9d\xbc\xeb\xb6\x81\xeb\x8f\x84 \xea\xb3\xb5\xea\xb3\xa0 \xec\xa0\x9c2011-791\xed\x98\xb8)\xec\x9d\x84 \xeb\xb0\x9b\xec\x95\x98\xec\x9c\xbc\xeb\xa9\xb0,&#13;\n\xed\x95\x9c\xea\xb5\xad\xea\xb4\x80\xea\xb4\x91\xea\xb3\xb5\xec\x82\xac \xea\xb5\xbf\xec\x8a\xa4\xed\x85\x8c\xec

In [13]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, 'lxml-xml')
items = soup.select('response > body > items > item')
for item in items:
  print(item.AREA.text,
        item.SNO.text,
        item.ADDRESS1.text,
        item.ADDRESS2.text,
        item.NAME.text,
        item.TEL.text,
        item.CATEGORY.text,
        )

12 1402 전북 전주시 덕진구 산정동 860-4 르시엘 호텔[우수숙박시설 굿스테이] 063-0245-4848 01
12 1401 전북 전주시 덕진구 덕진동2가  189-12 라뉘호텔[우수숙박시설 굿스테이] 063-0253-5708 01
12 1462 전북 전주시 덕진구 금암동  701-3 아리랑호텔[우수숙박시설 굿스테이] 063-0273-4193 01
12 1801 전북 전주시 완산구 고사동  227-32 우신호텔[우수숙박시설굿스테이] 063-0273-2232 01
12 1701 전북 전주시 완산구 다가동3가  43-2 풍남관광호텔 063-0231-7900 01


In [14]:
def parse():
  try:
    area = item.find('AREA').get_text()
    sno = item.find('SNO').get_text()
    address1 = item.find('ADDRESS1').get_text()
    address2 = item.find('ADDRESS2').get_text()
    name = item.find('NAME').get_text()
    tel = item.find('TEL').get_text()
    category = item.find('CATEGORY').get_text()

    return {
        "지역코드": area,
        "고유번호": sno,
        "주소" : address1,
        "상세주소" : address2,
        "이름" : name,
        "전화번호" : tel,
        "숙박종류" : category,
      }
  except AttributeError as e:
    return {
        "지역코드": None,
        "고유번호": None,
        "주소" : None,
        "상세주소" : None,
        "이름" : None,
        "전화번호" : None,
        "숙박종류" : None,

    }

In [15]:
row = []
for item in items:
  row.append(parse())

In [16]:
df = pd.DataFrame(row)
df

,지역코드,고유번호,주소,상세주소,이름,전화번호,숙박종류
0,12,1402,전북 전주시 덕진구 산정동,860-4,르시엘 호텔[우수숙박시설 굿스테이],063-0245-4848,01
1,12,1401,전북 전주시 덕진구 덕진동2가,189-12,라뉘호텔[우수숙박시설 굿스테이],063-0253-5708,01
2,12,1462,전북 전주시 덕진구 금암동,701-3,아리랑호텔[우수숙박시설 굿스테이],063-0273-4193,01
3,12,1801,전북 전주시 완산구 고사동,227-32,우신호텔[우수숙박시설굿스테이],063-0273-2232,01
4,12,1701,전북 전주시 완산구 다가동3가,43-2,풍남관광호텔,063-0231-7900,01
